In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression

In [ ]:
dir_train = '/kaggle/input/tabular-playground-series-may-2021/train.csv'
dir_test = '/kaggle/input/tabular-playground-series-may-2021/test.csv'
dir_sub = '/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv'

In [ ]:
data_train = pd.read_csv(dir_train, index_col = 'id')
data_test = pd.read_csv(dir_test, index_col = 'id')
data_sub = pd.read_csv(dir_sub, index_col = 'id')
data_train.head()

In [ ]:
data_train.feature_0.isnull().values.any()

In [ ]:
# Check if exists any column with null or NaN values
columns_with_nan_value = []
for column in data_train.columns:
    if(data_train[column].isnull().values.any()):
        columns_with_nan_value.append(column)
print(columns_with_nan_value)

In [ ]:
data_train.describe()

In [ ]:
data_train.columns

In [ ]:
data_train.dtypes

In [ ]:
data_train.target.hist()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_data = data_train.copy()

pure_cat_cols = data_train[data_train.columns].select_dtypes(exclude=['int64','float64']).columns
for col in pure_cat_cols:
    le.fit(le_data[col])
    le_data[col] = le.transform(le_data[col])
    #my_test_data[col] = le.transform(my_test_data[col])

corrdata = le_data
corrdata.head()

In [ ]:
correlation = corrdata.drop('target', axis=1).corrwith(corrdata.target).sort_values(axis=0, ascending=False)
correlation.head()

In [ ]:
correlation.tail()

In [ ]:
columns_high_correlation = ['feature_34', 'feature_36', 'feature_13', 'feature_2', 'feature_43', 'feature_29', 'feature_0', 'feature_14', 'feature_6', 'feature_37', 'target']

In [ ]:
from sklearn.preprocessing import LabelEncoder
import plotly.figure_factory as ff

primary_bgcolor = "#f4f0ea"

## correlation 
corr = corrdata[columns_high_correlation].corr().abs()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
corr1 = corr.mask(mask)

fig = ff.create_annotated_heatmap(
    z=corr1.to_numpy().round(2),
    x=list(corr1.index.values),
    y=list(corr1.columns.values),       
    xgap=3, ygap=3,
    zmin=0, zmax=1,
    colorscale='blugrn',
    colorbar_thickness=30,
    colorbar_ticklen=3,
)

fig.update_layout(
    title_text='<span style="font-size:32px; font-family:Times New Roman">Features Correlation Matrix</span>', 
    font_family="Serif",
    titlefont={'size': 24},
    width=800, height=700,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis_autorange='reversed', 
    paper_bgcolor=primary_bgcolor,
    plot_bgcolor=primary_bgcolor,
    margin=dict(l=70, r=70, t=70, b=70, pad=1),
)

fig.show()

In [ ]:
X = corrdata.drop('target', axis=1)
y = corrdata['target']

In [ ]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer

# X[X.columns] = QuantileTransformer().fit_transform(X)
# X[X.columns] = StandardScaler().fit_transform(X)
# X[X.columns] = RobustScaler().fit_transform(X)
X[X.columns] = PowerTransformer().fit_transform(X)

In [ ]:
X.head()

In [ ]:
len(X)

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
y_train.hist()

In [ ]:
y_test.hist()

In [ ]:
from sklearn.linear_model import SGDClassifier

sgdc = SGDClassifier(random_state=42)
sgdc.fit(X_train, y_train)
sgdc.score(X_test, y_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

In [ ]:
from sklearn.svm import LinearSVC

svc = LinearSVC(random_state=42)
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

randomForest = RandomForestClassifier(random_state=42)
randomForest.fit(X_train, y_train)
randomForest.score(X_test, y_test)

### Without upsampling:

#### QuantileTransformer:
- SGDClassifier: 0.5749
- KNeighborsClassifier: 0.51055
- LinearSVC: 0.5774
- RandomForestClassifier: 0.57555

#### PowerTransformer:
- SGDClassifier: 0.5752
- KNeighborsClassifier: 0.50375
- LinearSVC: 0.5774
- RandomForestClassifier: 0.576

#### RobustScaler
- SGDClassifier: 0.57525
- KNeighborsClassifier: 0.50565
- LinearSVC: 0.57735
- RandomForestClassifier: 0.5752

#### StandardScaler
- SGDClassifier: 0.57735
- KNeighborsClassifier: 0.5025
- LinearSVC: 0.5774
- RandomForestClassifier: 0.57415

In [ ]:
corrdata.target.hist()

In [ ]:
# Upsampling
class_with_more = corrdata[corrdata.target ==  1]
class_with_less  = corrdata[corrdata.target ==  0]
other_class = corrdata[corrdata.target >  1]

In [ ]:
# Agora vamos aplicar a técnica UpSampling, vamos importar a biblioteca sklearn.utils e escolher o resample.
from sklearn.utils import resample

len_class_with_more_samples = len(class_with_more)
class_with_less_upsample = resample(class_with_less, replace = True, n_samples = len_class_with_more_samples, random_state = 42)

In [ ]:
new_train_data = pd.concat([class_with_more, class_with_less_upsample, other_class])
new_train_data.target.hist()

In [ ]:
# Upsampling
class_with_more = new_train_data[new_train_data.target ==  0]
class_with_less  = new_train_data[new_train_data.target ==  2]
other_class_1 = new_train_data[new_train_data.target ==  1]
other_class_3 = new_train_data[new_train_data.target ==  3]

len_class_with_more_samples = len(class_with_more)
class_with_less_upsample = resample(class_with_less, replace = True, n_samples = len_class_with_more_samples, random_state = 42)

new_train_data = pd.concat([class_with_more, class_with_less_upsample, other_class_1, other_class_3])
new_train_data.target.hist()

In [ ]:
# Upsampling
class_with_more = new_train_data[new_train_data.target ==  0]
class_with_less  = new_train_data[new_train_data.target ==  3]
other_class_1 = new_train_data[new_train_data.target ==  1]
other_class_2 = new_train_data[new_train_data.target ==  2]

len_class_with_more_samples = len(class_with_more)
class_with_less_upsample = resample(class_with_less, replace = True, n_samples = len_class_with_more_samples, random_state = 42)

new_train_data = pd.concat([class_with_more, class_with_less_upsample, other_class_1, other_class_2])
new_train_data.target.hist()

In [ ]:
X = new_train_data.drop('target', axis=1)
y = new_train_data['target']

In [ ]:
data_sub.head()

In [ ]:
data_test.columns

In [ ]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer

# X[X.columns] = QuantileTransformer().fit_transform(X)
# X[X.columns] = StandardScaler().fit_transform(X)
# X[X.columns] = RobustScaler().fit_transform(X)
powerTransfomerFit = PowerTransformer().fit(X)
X[X.columns] = powerTransfomerFit.transform(X)
data_test = powerTransfomerFit.transform(data_test)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

randomForest = RandomForestClassifier(random_state=42)
randomForest.fit(X_train, y_train)
randomForest.score(X_test, y_test)

In [ ]:
# With upsampling
# PowerTransformer:
# - RandomForestClassifier: 0.9512370103047959

In [ ]:
predicts = randomForest.predict_proba(data_test)
predicts

In [ ]:
# With upsampling
# PowerTransformer:
# - RandomForestClassifier: 

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-may-2021/sample_submission.csv", index_col='id')
submission['Class_1']=predicts[:,0]
submission['Class_2']=predicts[:,1]
submission['Class_3']=predicts[:,2]
submission['Class_4']=predicts[:,3]
submission.to_csv('submission-tabular-playground-series-may-random-forest-30-05.csv')
submission.head()

In [ ]:
submission['Class_1'].hist()